## Data Load & Characteristics

- Load time series data and look at its characteristics

## XGBoost Model Creation & Forecasting


### Data Load 

### Create the XGBoost Model

### Forecast Using the XGBoost Model
